# Imports

In [65]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [11]:
test_path = "C:/Users/keanu/test_Y3wMUE5_7gLdaTN.csv"
train_path = "C:/Users/keanu/train_u6lujuX_CVtuZ9i.csv"

# Training set

In [12]:
ds_train = pd.read_csv(train_path)
ds_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


# Test set

In [13]:
ds_test = pd.read_csv(test_path)
ds_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


# Dataset shapes

In [15]:
print(f"training set (row, col): {ds_train.shape}\ntesting set (row, col): {ds_test.shape}")

training set (row, col): (614, 13)
testing set (row, col): (367, 12)


In [17]:
ds_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


# Dataset statistics

In [18]:
ds_train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [19]:
ds_train.drop('Loan_ID',axis=1,inplace=True)
ds_test.drop('Loan_ID',axis=1,inplace=True)
print(f"training set (row, col): {ds_train.shape}\ntesting set (row, col): {ds_test.shape}")

training set (row, col): (614, 12)
testing set (row, col): (367, 11)


In [20]:
ds_train.isnull().sum().sort_values(ascending=False)

Credit_History       50
Self_Employed        32
LoanAmount           22
Dependents           15
Loan_Amount_Term     14
Gender               13
Married               3
Education             0
ApplicantIncome       0
CoapplicantIncome     0
Property_Area         0
Loan_Status           0
dtype: int64

In [21]:
null_cols = ['Credit_History', 'Self_Employed', 'LoanAmount','Dependents', 'Loan_Amount_Term', 'Gender', 'Married']
for col in null_cols:
    ds_train[col] = ds_train[col].fillna(ds_train[col].dropna().mode().values[0])

In [22]:
ds_train.isnull().sum().sort_values(ascending=False)

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [23]:
#converting categorical values to numbers

to_numeric = {'Male': 1, 'Female': 2,
'Yes': 1, 'No': 2,
'Graduate': 1, 'Not Graduate': 2,
'Urban': 3, 'Semiurban': 2,'Rural': 1,
'Y': 1, 'N': 0,
'3+': 3}

# adding the new numeric values from the to_numeric variable to both datasets
ds_train = ds_train.applymap(lambda lable: to_numeric.get(lable) if lable in to_numeric else lable)
ds_test = ds_test.applymap(lambda lable: to_numeric.get(lable) if lable in to_numeric else lable)

# convertind the Dependents column
Dependents_ = pd.to_numeric(ds_train.Dependents)
Dependents__ = pd.to_numeric(ds_test.Dependents)

# dropping the previous Dependents column
ds_train.drop(['Dependents'], axis = 1, inplace = True)
ds_test.drop(['Dependents'], axis = 1, inplace = True)

# concatination of the new Dependents column with both datasets
ds_train = pd.concat([ds_train, Dependents_], axis = 1)
ds_test = pd.concat([ds_test, Dependents__], axis = 1)

# checking the our manipulated dataset for validation
print(f"training set (row, col): {ds_train.shape}\n\ntesting set (row, col): {ds_test.shape}\n")
print(ds_train.info(), "\n\n", ds_test.info())

training set (row, col): (614, 12)

testing set (row, col): (367, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             614 non-null    int64  
 1   Married            614 non-null    int64  
 2   Education          614 non-null    int64  
 3   Self_Employed      614 non-null    int64  
 4   ApplicantIncome    614 non-null    int64  
 5   CoapplicantIncome  614 non-null    float64
 6   LoanAmount         614 non-null    float64
 7   Loan_Amount_Term   614 non-null    float64
 8   Credit_History     614 non-null    float64
 9   Property_Area      614 non-null    int64  
 10  Loan_Status        614 non-null    int64  
 11  Dependents         614 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 57.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 11 column

In [ ]:
#plotting the correlation matrix
sns.heatmap(ds_train.corr() ,cmap='cubehelix_r')

#sns.pairplot(ds_train.corr(), hue = 'diagnosis', diag_kind='kde');

# Sequential Model

In [67]:
y_train = ds_train['Loan_Status']
X_train = ds_train.drop('Loan_Status', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [72]:
model = Sequential()
model.add(Dense(units=12, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [73]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [74]:
model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
14/14 [==============================] - 0s 1ms/step - loss: 107365.6484 - accuracy: 0.5874
Epoch 2/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6363 - accuracy: 0.6713
Epoch 3/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6359 - accuracy: 0.6713
Epoch 4/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6392 - accuracy: 0.6713
Epoch 5/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6371 - accuracy: 0.6713
Epoch 6/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6392 - accuracy: 0.6713
Epoch 7/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6332 - accuracy: 0.6713
Epoch 8/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6358 - accuracy: 0.6713
Epoch 9/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6371 - accuracy: 0.6713
Epoch 10/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6349 - accuracy: 0.6713
Epoc

Epoch 83/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6379 - accuracy: 0.6713
Epoch 84/200
14/14 [==============================] - 0s 4ms/step - loss: 0.6402 - accuracy: 0.6713
Epoch 85/200
14/14 [==============================] - 0s 5ms/step - loss: 0.6382 - accuracy: 0.6713
Epoch 86/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6365 - accuracy: 0.6713
Epoch 87/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6385 - accuracy: 0.6713
Epoch 88/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6358 - accuracy: 0.6713
Epoch 89/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6359 - accuracy: 0.6713
Epoch 90/200
14/14 [==============================] - 0s 4ms/step - loss: 0.6355 - accuracy: 0.6713
Epoch 91/200
14/14 [==============================] - 0s 4ms/step - loss: 0.6374 - accuracy: 0.6713
Epoch 92/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6370 - accuracy: 0.6713


14/14 [==============================] - 0s 1ms/step - loss: 0.6362 - accuracy: 0.6713
Epoch 165/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6377 - accuracy: 0.6713
Epoch 166/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6420 - accuracy: 0.6713
Epoch 167/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6358 - accuracy: 0.6713
Epoch 168/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6359 - accuracy: 0.6713
Epoch 169/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6356 - accuracy: 0.6713
Epoch 170/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6380 - accuracy: 0.6713
Epoch 171/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6346 - accuracy: 0.6713
Epoch 172/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6393 - accuracy: 0.6713
Epoch 173/200
14/14 [==============================] - 0s 1ms/step - loss: 0.6396 - accuracy: 0.6713
Epoc

In [75]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

6/6 [==============================] - 0s 2ms/step


In [76]:
accuracy_score(y_test, y_hat)

0.7243243243243244

In [47]:
print(y_hat)

[[0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]
 [0.6887149]

# Decision Tree Model

In [71]:
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)

y_predict = DT.predict(X_test)

#  prediction Summary by species
print(classification_report(y_test, y_predict))

# Accuracy score
DT_SC = accuracy_score(y_predict,y_test)
print(f"{round(DT_SC*100,2)}% Accurate")

              precision    recall  f1-score   support

           0       0.44      0.59      0.50        51
           1       0.82      0.72      0.76       134

    accuracy                           0.68       185
   macro avg       0.63      0.65      0.63       185
weighted avg       0.72      0.68      0.69       185

68.11% Accurate
